### As always, import the necessary packages

In [30]:
#-----Utils-----
from pyimagesearch.siamese_network import build_siamese_model
from pyimagesearch import utils
#-----Tensorflow-----
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Dense, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
#-----Others-------
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras.backend as K
import cv2

### Configuration options

In [2]:
# specify the shape of the inputs for our network
IMG_SHAPE = (28, 28, 1)

# specify the batch size and number of epochs
BATCH_SIZE = 64
EPOCHS = 10

# define the path to the base output directory
BASE_OUTPUT = "output"
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

### Get the data and preprocess

In [3]:
(trainX, trainY), (testX, testY) = mnist.load_data()

#Scaling the images
trainX = trainX / 255.0
testX = testX / 255.0

#Add extra channel. This is due to the make_pairs function (check utils.py for more details)
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

### Get the pairs

In [4]:
(pairTrain, labelTrain) = utils.make_pairs(trainX, trainY)
(pairTest, labelTest) = utils.make_pairs(testX, testY)

### Configure the Siamese network and the inputs/outputs

In [5]:
#Define the inputs (image A and B) of the Siamese network
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)

#We initialize the network but we take it as a feature extractor. This is to obey one of the principles pf Siamese Networks: same weights updates
#If we give first imgA as input and then imgB then the weights will be different
feature_extractor = build_siamese_model(IMG_SHAPE)
featsA = feature_extractor(imgA)
featsB = feature_extractor(imgB)

2022-09-16 12:25:16.280511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 12:25:16.338533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 12:25:16.339122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 12:25:16.341418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [35]:
#Once we have the features, we pass those through the euclidian distance (to calculate similarity) and then into the Dense Layer to get the output
#This is using the functional API
distance = Lambda(utils.euclidean_distance)([featsA, featsB])
#outputs = Dense(1, activation= "sigmoid")(distance) We have to use the distance itself as the output.If not, the loss will be the same of both train and test
model = Model(inputs=[imgA, imgB], outputs=distance)

### Define the contrastive loss

In [36]:
def contrastive_loss(y, preds, margin=1):
	# explicitly cast the true class label data type to the predicted class label data type
	y = tf.cast(y, preds.dtype)

	# calculate the contrastive loss between the true labels and the predicted labels
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

	# return the computed contrastive loss to the calling function
	return loss

### Compile the model and train it

In [37]:
model.compile(loss=contrastive_loss, optimizer="adam")

In [38]:
#Give the positive images (1) and negatives(0) into a nested list
history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:], validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
	batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2665 - val_loss: 0.2675
Epoch 2/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2354 - val_loss: 0.2589
Epoch 3/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2320 - val_loss: 0.2696
Epoch 4/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2310 - val_loss: 0.2624
Epoch 5/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2284 - val_loss: 0.2512
Epoch 6/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.2267 - val_loss: 0.2575
Epoch 7/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.2255 - val_loss: 0.2547
Epoch 8/10
1875/1875 [==============================] - 22s 11ms/step - loss: 0.2249 - val_loss: 0.2458
Epoch 9/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.2242 - val_loss: 0.2545
Epoch 10/10
1875/1875 [==============================] - 22s 12m

### We get lower loss

In [39]:
model.save(MODEL_PATH)

INFO:tensorflow:Assets written to: output/siamese_model/assets


INFO:tensorflow:Assets written to: output/siamese_model/assets
